In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from server_pull import pull
import random
import math

# Make sure server_pull.py file is in the same working directory
import os
os.getcwd()

In [ ]:
NetReward = 0
NetPull = 0
TOT_MACHINE = 24

def test_pull(user_group, secret_key, machine_num):
    global NetReward
    global NetPull
    
    NetReward = NetReward + 1
    NetPull = NetPull + 1
    
    return {'Arm': str(machine_num), 'NetReward' : NetReward, 'Pull' : NetPull, 'Reward': np.random.randint(0, 100)}

In [ ]:
#output = pull('user32','cnERvnBr',2)
#output = pull('user40','XEq6wByD',2)

output = test_pull('test_user','aaaaaaaa',23)
output

In [ ]:
# Function that decreases linearly with T

def epsilon_decay_function(initial_epsilon, min_epsilon, epsilon_decay, timesteps):
    epsilon_values = []
    epsilon = initial_epsilon

    for _ in timesteps:
        epsilon = max(min_epsilon, epsilon * epsilon_decay)
        epsilon_values.append(epsilon)

    return epsilon_values

# 매개변수 설정
initial_epsilon = 1.0  # 시작 ε 값
min_epsilon = 0.01     # 최소 ε 값
epsilon_decay = 0.995  # ε 감소율
timesteps = range(100) # 시간 단계 범위

# 함수 실행 및 결과 확인
epsilon_values = epsilon_decay_function(initial_epsilon, min_epsilon, epsilon_decay, timesteps)
epsilon_values

# 결과 시각화
plt.figure(figsize=(10, 6))
plt.plot(timesteps, epsilon_values, label='Epsilon Decay')
plt.xlabel('Timestep')
plt.ylabel('Epsilon Value')
plt.title('Epsilon Decay Over Time')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Function that decreases exponentialy with T
import matplotlib.pyplot as plt
import math

# Define the epsilon_t function with exponential decay
def epsilon_t_exp(t, epsilon_0=1.0, k=0.01):
    return 1/t
    # return epsilon_0 * math.exp(-k * t)


# Generate data for the plot
timesteps = range(1,1001)  # 0 to 99
values = [epsilon_t_exp(t) for t in timesteps]

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(timesteps, values, label='Epsilon(t)')
# plt.plot(timesteps, values, label='Epsilon(t)')
plt.xlabel('Timestep')
plt.ylabel('Epsilon')
plt.title('Epsilon Decay Over Time')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# 최종 테스트

def adaptive_epsilon_greedy(user_group, password, given_rounds):
    # Initialize the result dict with arms and statistics
    result = {idx: {
                    "Arm": idx,
                    "Output_Arm": 0,
                    "N_List": [], # Rewards gotten during Exploration
                    "Reward_List": [], # Rewards obtained throughout rounds
                    "Round_List": [], # Rounds which the arm is pulled
                    } for idx in range(TOT_MACHINE)
             }
    
    # Epsilon Value Trend 
    # epsilons = []  # List of Chaning Epsilon Value (입실론의 변화를 추적하기 위한 리스트)
    
    # for num_round in range(1, given_rounds):
    # Update epsilon using the new calculation method
    # epsilon = epsilon_t_exp(num_round)

    # Define the epsilon_t function with exponential decay
    def epsilon_t_exp(t, epsilon_0=1.0, k=0.01): # k 값은 Decaying Value. epsilon_0 초기 값
        # return 1/t 
        return epsilon_0 * math.exp(-k * t)
    
    for num_round in range(1, given_rounds):
        current_epsilon = epsilon_t_exp(num_round)


    # Decide for Exploitation / Exploration
    for num_round in range(1, given_rounds):
        current_epsilon = epsilon_t_exp(num_round)
        print(f"Round {num_round}: current_epsilon = {current_epsilon}")
        # current_epsilon = epsilon_t_exp(num_round)
        # rand_value = random.random()
        # print(f"Round: {num_round}, Epsilon: {current_epsilon}, Random Value: {rand_value}")

        if random.random() < current_epsilon: # Decide whether to explore or exploit
            arm = random.randint(1, TOT_MACHINE - 1)
            print(f"# {num_round}, Exploring Arm {arm}...")
            output = pull(user_group, password, arm) # Simulate pulling an arm and getting an output for exploration 해당 부분 추가
            result[arm]["N_List"].append(output["Reward"]) # Append data to N_List for exploration # 해당 부분 추가

        # Exploit: choose the best machine so far
        else:
            arm = max(result, key=lambda k: sum(result[k]["Reward_List"]) / len(result[k]["Reward_List"]) if len(result[k]["Reward_List"]) > 0 else 0) # ==> Here is the problem. 이 부분 수정 혹은 바로 아래의 코드
            #arm = max(result, key=lambda k: sum(result[k]["Reward_List"]) / len(result[k]["Reward_List"]))
            output = pull(user_group, password, arm) # Simulate pulling an arm and getting an output for exploitation # 해당 부분 추가.
            print(f"# {num_round}, Exploiting Arm {arm}...")


###################################################################################################
    #     # Decide whether to explore or exploit
    #     if random.random() <= epsilon:
    #         # Explore: randomly choose an arm
    #         arm = random.randint(0, TOT_MACHINE)
    #         print(f"# {num_round}, Exploring Arm {arm}...")
    #     else:
    #         # Exploit: choose the arm with the highest average reward
    #         arm = max(result, key=lambda k: sum(result[k]["Reward_List"]))
    #         print(f"# {num_round}, Exploiting Arm {arm}...")
###################################################################################################

        # Simulate pulling an arm and getting an output
        # Assuming pull is a function that you've defined elsewhere
        # output = pull(user_group, password, arm)
        
        # Append Data
        result[arm]["Output_Arm"] = output["Arm"]
        result[arm]["Reward_List"].append(output["Reward"])
        result[arm]["Round_List"].append(num_round)


    list_result = [
        {**v, 
        "Total_Reward": sum(v["Reward_List"]),
        "Avg_N": sum(v["N_List"]) / len(v["N_List"]) if len(v["N_List"]) > 0 else 0, # if len(v["N_List"]) > 0 else 0 추가.
        "Avg_Reward": sum(v["Reward_List"]) / len(v["Reward_List"]) if len(v["Reward_List"]) > 0 else 0 # if len(v["Reward_List"]) > 0 else 0 추가
        } for v in result.values()
    ]
    return list_result

###################################################################################################

# Regret Function
def regret(result, given_rounds):
    best_result = max(result, key=lambda x:len(x["Reward_List"]))
    
    print(f"Best Arm Identified: {best_result}\n")
    
    x = [i for i in range(1, given_rounds + 1)]
    y = []
    
    best_mean = sum(best_result["Reward_List"])/len(best_result["Reward_List"])
    
    for num_round in range(1, given_rounds + 1):        
        indices = [idx for idx, round_num in enumerate(best_result["Round_List"]) if round_num <= num_round]
        tot_rewards = sum([best_result["Reward_List"][idx] for idx in indices])
        
        y.append((best_mean * num_round) - tot_rewards)
        
    # Graph Plot
    plt.plot(x,y)
    plt.xlabel("No. of Rounds")
    plt.ylabel("Total Regret")
    plt.show()

In [ ]:
# 실제 ARM 당기기

#output = pull('user32','cnERvnBr',2)
#output = pull('user40','XEq6wByD',2)

given_rounds = 1000

result = adaptive_epsilon_greedy("user40", "XEq6wByD", given_rounds + 1)

print("\n##########################################################################################")
print("Total Runs")
print("##########################################################################################\n")
for elem in result:
    print(elem)

print("\n##########################################################################################")
print("Regret Plot")
print("##########################################################################################\n")
regret(result, 1000)

In [ ]:

print("\n##########################################################################################")
print("Regret Plot")
print("##########################################################################################\n")
regret(result, 1000)

In [ ]:
# Regret Function
def save_csv(result, given_rounds):
    best_result = max(result, key=lambda x:len(x["Reward_List"]))
    
    print(f"Best Arm Identified: {best_result}\n")
    
    x = [i for i in range(1, given_rounds + 1)]
    y = []
    
    best_mean = sum(best_result["Reward_List"])/len(best_result["Reward_List"])
    
    for num_round in range(1, given_rounds + 1):        
        indices = [idx for idx, round_num in enumerate(best_result["Round_List"]) if round_num <= num_round]
        tot_rewards = sum([best_result["Reward_List"][idx] for idx in indices])
        
        y.append((best_mean * num_round) - tot_rewards)
    data = pd.DataFrame(y,x)
    # return data
    data.to_csv('exponential_one.csv')
    # # Graph Plot
    # plt.plot(x,y)
    # plt.xlabel("No. of Rounds")
    # plt.ylabel("Total Regret")
    # plt.show()

In [ ]:
save_csv(result, 1000)